In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:


warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import string
import math
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


from sklearn import metrics

In [ ]:
train_data = pd.read_csv("../input/health-insurance-cross-sell-prediction/train.csv")
test_data = pd.read_csv("../input/health-insurance-cross-sell-prediction/test.csv")

In [ ]:
print(train_data.shape)
print(test_data.shape)

print('Features : ', train_data.columns.values)
print('Features : ', test_data.columns.values)
train_data.head()

In [ ]:
train_data.info()

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

In [ ]:
train_data.nunique()

In [ ]:
train_data["Response"].value_counts(normalize= True)

In [ ]:
import seaborn as sns

sns.countplot(train_data["Response"])

# **Exploratory Data Analysis**

In [ ]:
sns.countplot(x ='Gender', hue='Response',data = train_data)

In [ ]:
sns.countplot(x="Gender", data = train_data)

In [ ]:
sns.countplot(x="Driving_License", data = train_data)

In [ ]:
sns.countplot(x="Previously_Insured", data = train_data)

In [ ]:
sns.distplot(train_data.Age)

In [ ]:
sns.countplot(x="Vehicle_Damage", data = train_data)

In [ ]:
sns.countplot(x="Vehicle_Age", data = train_data)

In [ ]:
sns.countplot(x="Response", data = train_data)

# DATA PREPROCESSING

In [ ]:

train = train_data.copy()
test = test_data.copy()

In [ ]:
def preprocessing(data):
    
    #Dropping null values
    data = data.dropna()
    
    #Dropping id
    data = data.drop('id', axis = 1)
    
    #Columns to get dummies
    cols = ['Gender', 'Vehicle_Damage', 'Vehicle_Age']
    
    #Changing categories into dummies
    data_dum = pd.get_dummies(data = data , columns = cols, drop_first = True )
    
    #We don't need this column as it has almost no correlation with our dependent variable
    data_dum = data_dum.drop('Vintage', axis = 1)
    
    return data_dum

In [ ]:
#Preprocessing training data
train_dum = preprocessing(train)

#Preprocessing test data
test_dum = preprocessing(test)


In [ ]:
x_test.info()

# **Splitting data(Imbalanced data)**

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 10, random_state = 42)

for train_idx, val_idx in skf.split(inputs, targets):
    x_train, x_val = inputs.iloc[train_idx], inputs.iloc[val_idx]
    y_train, y_val = targets.iloc[train_idx], targets.iloc[val_idx]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

# **Modelling**

In [ ]:
Results = pd.DataFrame(columns = ['Model', 'Validation Score'])

Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth=7, n_estimators = 500)

rf.fit(x_train_scaled,y_train)

y_val_pred = rf.predict_proba(x_val_scaled)[:,1]

score = roc_auc_score(y_val, y_val_pred)
print(score)

In [ ]:

Results = Results.append({'Model' : 'Random Forest', 'Validation Score': score}, ignore_index = True)

XGBoost

In [ ]:
from xgboost import XGBClassifier

xg = XGBClassifier(random_state = 42, learning_rate = 0.1)

xg.fit(x_train_scaled, y_train)

y_val_pred = xg.predict_proba(x_val_scaled)[:,1]

score = roc_auc_score(y_val, y_val_pred)
print(score)

In [ ]:
Results = Results.append({'Model' : 'XGBoost', 'Validation Score': score}, ignore_index = True)

In [ ]:
CatBoost

In [ ]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier()

cat.fit(x_train_scaled, y_train, plot=True, early_stopping_rounds=30, verbose=100)

y_val_pred = cat.predict_proba(x_val_scaled)[:,1]

score = roc_auc_score(y_val, y_val_pred)
print(score)

In [ ]:
Results = Results.append({'Model' : 'CatBoost', 'Validation Score': score}, ignore_index = True)

LGBMClassifier

In [ ]:


from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

lgbm = LGBMClassifier(num_leaves = 30, max_depth = 5, n_estimators = 550, learning_rate = 0.05, objective = 'binary', 
                      lambda_l2 = 12,
                      max_bin = 100, metric = 'auc', is_unbalance = True, random_state = None, n_jobs = -1)

lgbm.fit(x_train_scaled,y_train)

y_val_pred = lgbm.predict_proba(x_val_scaled)[:,1]

score = roc_auc_score(y_val, y_val_pred)
print(score)

In [ ]:
Results = Results.append({'Model' : 'LGBM', 'Validation Score': score}, ignore_index = True)

In [ ]:
Results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

In [ ]:
plt.figure(figsize = (10,6))
sns.barplot(data = Results, y = 'Validation Score', x = 'Model')
plt.ylim(0.84,0.86)
plt.show()

In [ ]:
y_pred = lgbm.predict_proba(x_test_scaled)[:,1]

In [ ]:
Response = pd.DataFrame({'id': test.id, 'Response': y_pred})
print(Response)
